In [36]:
MODEL = "deepseek-r1:1.5b"

In [ ]:
import pymupdf  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Example usage
pdf_text = extract_text_from_pdf("your_directory\PASAKAnimeArticle.pdf")
pdf_text

"See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/319494624\nStudy on Anime and Its Impacts Among University Students\nArticle · April 2016\nCITATIONS\n4\nREADS\n168,291\n3 authors:\nNor Azlili Hassan\nTunku Abdul Rahman University\n44 PUBLICATIONS\xa0\xa0\xa0171 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nIza SHARINA Sallehuddin\nTunku Abdul Rahman University\n11 PUBLICATIONS\xa0\xa0\xa046 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nNik Norazira Abdul Aziz\nTunku Abdul Rahman University\n9 PUBLICATIONS\xa0\xa0\xa011 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Nor Azlili Hassan on 06 September 2017.\nThe user has requested enhancement of the downloaded file.\nA Study On Anime and Its Impacts Among University Students \nNor Azlili binti Hassan1a, Iza Sharina bt Sallehuddin2, and Nik Norazira bt Abd Aziz3 \n1Department of General Studies, Universiti Tunku Abdul Rahman (UTAR), Sg Long Campus, Malaysia \n2

In [38]:
import re

def preprocess_text(text):
    # Remove extra whitespace and special characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

cleaned_text = preprocess_text(pdf_text)

In [39]:
def preprocess_text(text, chunk_size=512):
    # Basic cleaning (remove extra spaces, etc.)
    text = " ".join(text.split())

    # Split text into chunks
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    return chunks

# Example usage
text_chunks = preprocess_text(pdf_text)
text_chunks[0]

'See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/319494624 Study on Anime and Its Impacts Among University Students Article · April 2016 CITATIONS 4 READS 168,291 3 authors: Nor Azlili Hassan Tunku Abdul Rahman University 44 PUBLICATIONS 171 CITATIONS SEE PROFILE Iza SHARINA Sallehuddin Tunku Abdul Rahman University 11 PUBLICATIONS 46 CITATIONS SEE PROFILE Nik Norazira Abdul Aziz Tunku Abdul Rahman University 9 PUBLICATIONS 11 CITATIONS SEE PROFIL'

In [40]:
import ollama
embeddings = [ollama.embeddings(model=MODEL, prompt=text)["embedding"] for text in text_chunks]
embeddings

[[-1.362311840057373,
  1.8611007928848267,
  -0.7087163925170898,
  0.018631089478731155,
  -0.1063622459769249,
  2.074035167694092,
  -1.0891976356506348,
  1.1490216255187988,
  1.4258382320404053,
  -1.590570330619812,
  0.7182300686836243,
  1.2076658010482788,
  1.2373170852661133,
  -1.3322100639343262,
  1.4528082609176636,
  -1.247316598892212,
  -0.11451543122529984,
  -0.5542023181915283,
  -0.2009400576353073,
  0.8500863909721375,
  -0.47844409942626953,
  -3.046494483947754,
  0.6902398467063904,
  0.4078328311443329,
  -1.0674055814743042,
  2.465196371078491,
  -1.9947822093963623,
  0.8488335609436035,
  -1.7396035194396973,
  -0.17211486399173737,
  1.9389954805374146,
  2.7115590572357178,
  -3.3485286235809326,
  1.8972991704940796,
  0.5887632966041565,
  0.10449021309614182,
  -2.5613317489624023,
  0.4913386404514313,
  3.12142276763916,
  -2.808784246444702,
  1.2721805572509766,
  4.672354698181152,
  0.12541253864765167,
  -1.168855905532837,
  0.745892107486

In [ ]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
embeddings_np = np.array(embeddings, dtype='float32')

# Create a FAISS index (L2 distance)
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

# Save FAISS index to a file (optional)
faiss.write_index(index, "faiss_index.bin")

print("FAISS index created and saved.")

FAISS index created and saved.


In [31]:
def retrieve_relevant_chunks(query, top_k=2):
    """Retrieve the most relevant text chunks using DeepSeek embeddings."""
    query_embedding = ollama.embeddings(model=MODEL, prompt=query)["embedding"]
    query_np = np.array([query_embedding], dtype='float32')

    # Search in FAISS index
    D, I = index.search(query_np, k=top_k)
    
    # Retrieve matching text chunks
    relevant_chunks = [text_chunks[i] for i in I[0]]
    return relevant_chunks

# Example Query
query = "How does DeepSeek work?"
retrieved_chunks = retrieve_relevant_chunks(query)
print("🔍 Retrieved Chunks:", retrieved_chunks)


🔍 Retrieved Chunks: ['.3%(22) 44.2% (76) 51% (38) 37% (63) Enjoy watching anime programmes Male Female 75 172 24% (18) 30% (51) 24% (18) 34.9% (60) 52% (39) 36% (61) Watching anime programmes give feeling of satisfaction Male Female 75 172 23% (17) 36% (62) 34.7% (26) 34.9% (60) 43% (32) 29% (50) Will never miss watching anime programme every week Male Female 75 172 53% (40) 63% (108) 30.7% (23) 25.6% (44) 16% (12) 13% (20) Enjoy watching characters in every episode shown in anime Male Female 75 172 29% (22) 49% (84) 34.7% (26) ', 'n. Asian Journal of Communication, 22(1), 44-57. Drazen, P. (2003). Anime Explosion: The What? Why? & Wow! of Japanese Animation. California: Stone Bridge Press. Elza, C. (2009). We all Live in a Pokemon World: Animated Utopia for Kids. In: WEST, Mark (Ed.). The Japanification of Children‟s Popular Culture. Maryland: Scarecrow Press. Erikson, E. (1968). Identity Youth and Crisis. New York: Norton. Fennell, D., Ana S.Q. Liberato, Hayden, B. (2012). Consuming 

In [35]:
def generate_response(query):
    """Retrieve relevant context and generate a response using DeepSeek."""
    relevant_chunks = retrieve_relevant_chunks(query)
    context = "\n".join(relevant_chunks)

    prompt = f"Answer the following query based on the provided context:\n\nContext: {context}\n\nQuery: {query}\nAnswer:"
    
    response = ollama.chat(model=MODEL, messages=[{"role": "user", "content": prompt}])
    
    return response["message"]["content"]

# Example query
query = "Difference of initial attitudes towards anime between genders"
response = generate_response(query)
print("🤖 AI Response:", response)


🤖 AI Response: <think>
Alright, I need to figure out how to answer this query based on the provided context. The user's query is "Difference of initial attitudes towards anime between genders," which seems straightforward but I should verify that.

Looking at the context, I see a table labeled "Query:" with several columns and rows of percentages for different demographics regarding anime collection, merchandising, etc. There are multiple sections, each under different categories like Male Female, 75, 172, and various percentages. Each row has data about how people perceive anime in terms of being collected (positive), merchandising (negative), etc.

The query is asking for the difference between initial attitudes towards anime between genders. The context does mention that there are sections under "Male Female" which detail the perceptions of both males and females regarding anime collection, merchandising, etc. These provide numerical data on positive and negative percentages for eac